In [1]:
import torch


In [2]:
x = torch.arange(4.0)
print(x)

tensor([0., 1., 2., 3.])


In [34]:
#二者等价
x = torch.arange(4.0, requires_grad=True)
x.requires_grad_(True)
print(x.grad)   #默认为None

None


## y为标量的情况

In [35]:
y = 2*torch.dot(x,x)        #只能1D
print(y)

tensor(28., grad_fn=<MulBackward0>)


In [36]:
y.backward()
print(x.grad)

tensor([ 0.,  4.,  8., 12.])


In [37]:
print(x.grad == 4*x)

tensor([True, True, True, True])


In [38]:
#计算sum函数对x的梯度
x.grad.zero_()  #梯度会累积，需要清楚之前的值
y = x.sum()
y.backward()
print(x.grad)


tensor([1., 1., 1., 1.])


## 非标量变量的反向传播
当y不是标量时，向量y关于向量x的导数的最自然解释是一个矩阵。 对于高阶和高维的y和x，求导的结果可以是一个高阶张量


In [39]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 在我们的例子中，我们只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
print(x.grad)

tensor([0., 2., 4., 6.])


如果y不是标量，需要在backward里设置参数，tensor大小和y一样

y向量m*1对x向量n*1的求导是一个矩阵m*n，backword里需要放一个和x向量同形状的tensor，对矩阵求加权和
一般用都是1的tensor,最后返回m*1的向量

In [ ]:
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.backward(torch.ones(len(x)))
print(x.grad)

## 分离计算
有时，我们希望将某些计算移动到记录的计算图之外。 例如，假设y是作为x的函数计算的，而z则是作为y和x的函数计算的。 想象一下，我们想计算z关于x的梯度，但由于某种原因，我们希望将y视为一个常数， 并且只考虑到x在y被计算后发挥的作用。

在这里，我们可以分离y来返回一个新变量u，该变量与y具有相同的值， 但丢弃计算图中如何计算y的任何信息。 换句话说，梯度不会向后流经u到x。 因此，下面的反向传播函数计算z=u*x关于x的偏导数，同时将u作为常数处理， 而不是z=x*x*x关于x的偏导数。

In [42]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u


tensor([True, True, True, True])

In [43]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x


tensor([True, True, True, True])

In [47]:
def f(a):
    b = a * 2
    while b.norm() < 1000:      #L2范数
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [50]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

In [49]:
a.grad == d/a

tensor(True)